In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [59]:
df = pd.read_csv("WineQT.csv", sep=",")
df = df.drop(columns="Id")
# print(df.describe())

In [60]:
def plot_hists(df):
    fig, ax = plt.subplots(nrows=3, ncols=4, figsize=(20, 15))
    for n in range(12):
        i = n % 3
        j = n % 4
        ax[i, j].hist(df.iloc[:, n], bins='auto')
        ax[i, j].set_xlabel(df.columns[n])

In [61]:
# plot_hists(df)

In [62]:
def normalize(df, property, parameter):
    df[property] = np.log(df[property] + parameter)

normalize(df, "fixed acidity", -2.3)
normalize(df, "sulphates", -0.24)
normalize(df, "total sulfur dioxide", 5)
normalize(df, "residual sugar", -1.1)
normalize(df, "chlorides", -0.005)
normalize(df, "volatile acidity", 2)
normalize(df, "free sulfur dioxide", 2)
# plot_hists(df)

C:\Users\ktabakh\AppData\Roaming\Python\Python39\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [63]:
standardized = (df - df.mean()) / df.std()
standardized = standardized[(np.abs(standardized) < 3).all(axis=1)]
rows = np.setdiff1d(list(df.index), list(standardized.index))
df.drop(index=rows, inplace=True)
# plot_hists(standardized)

In [64]:
# sns.pairplot(standardized, corner=True)

In [65]:
corr = standardized.corr()
# matrix = np.triu(corr, 1)
# fig, ax = plt.subplots(figsize=(10, 10))
# ax = sns.heatmap(corr, annot=True, fmt=".3f", mask=matrix)

In [66]:
np.fill_diagonal(corr.values, 0)
def delete(corr, df):
    tmp = corr.loc[:, (np.abs(corr) >= 0.7).any()]
    print((tmp.columns))
    if (len(tmp.columns) > 0):
        label = np.abs(tmp).sum().sort_values(ascending=False).index[0]
        corr.drop(index=label, columns=label, inplace=True)
        df.drop(columns=label, inplace=True)
        delete(corr, df)
delete(corr, df)
# print(df.describe())

Index(['fixed acidity', 'free sulfur dioxide', 'total sulfur dioxide', 'pH'], dtype='object')
Index(['free sulfur dioxide', 'total sulfur dioxide'], dtype='object')
Index([], dtype='object')


In [67]:
# from sklearn.model_selection import train_test_split

# X = np.c_[df.iloc[:, :-1].to_numpy(), np.ones(df.shape[0])]
# y = df["quality"].to_numpy()

# X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.4, random_state=42)
# X_cv, X_test, y_cv, y_test = train_test_split(X_tmp, y_tmp, test_size=0.5, random_state=42)

In [68]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler().fit(X_train)

# X_train = scaler.transform(X_train)
# X_cv = scaler.transform(X_cv)
# X_test = scaler.transform(X_test)

# w = np.ones(X.shape[1]).reshape(-1, 1)
# f = lambda X: np.matmul(X, w)